# visualization

In [ ]:
! pip install spectral
! pip install plotly
! pip install pathlib
! pip install spectral
! pip install numpy
! pip install opencv-python
! pip install tqdm

In [3]:
from spectral import *
from  pathlib import Path
import cv2 as cv
import numpy as np

data_dir = r'E:\PHD\Blueberry\Blueberry\Blueberry_Pretest'

names = [ 'Blueberry_Pretest_Sample000_05112022', 'Blueberry_Pretest_Dark_05112022', 'Blueberry_Pretest_White_05112022']
path = Path(data_dir) / f'{names[0]}.bil'.__str__()
path_head = Path(data_dir) / f'{names[0]}.bil.hdr'.__str__()

ref_path = Path(data_dir) / f'{names[1]}.bil'.__str__()
ref_path_head = Path(data_dir) / f'{names[1]}.bil.hdr'.__str__()

ref1_path = Path(data_dir) / f'{names[2]}.bil'.__str__()
ref1_path_head = Path(data_dir) / f'{names[2]}.bil.hdr'.__str__()

In [4]:
from spectral.io import envi
"""
    Returns:

        :class:`spectral.SpyFile` or :class:`spectral.io.envi.SpectralLibrary`
        object.
"""
data = envi.open(path_head, path)
ref_data = envi.open(ref_path_head, ref_path)
ref1_data = envi.open(ref1_path_head, ref1_path)

In [3]:
data#, ref_data

	Data Source:   'E:\PHD\Blueberry\Blueberry\Blueberry_Pretest_Sample000_05112022.bil'
	# Rows:           1000
	# Samples:        1600
	# Bands:           462
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16

In [ ]:
import plotly.express as px
import numpy as np

im = data[:, :, 100]
im_np = np.array(im/im.max()*255, dtype=np.uint8).squeeze()
print(im_np.shape)
fig = px.imshow(im_np)
fig.show()
data.shape[2]

In [ ]:
import plotly.express as px
import numpy as np

taget_y_0 = 119
taget_y_1 = 290
taget_x_0 = 162
taget_x_1 = 350

im = im_np[taget_y_0:taget_y_1, taget_x_0:taget_x_1]
fig = px.imshow(im)
fig.show()
data.shape[2]

In [5]:
data
ref_data
band_num = data.shape[2]
im_mins = [ ]
im_maxs= [ ]
ref_mins= [ ]
ref_maxs= [ ]
for idx in range(band_num):
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    im_mins.append(im.min())
    im_maxs.append(im.max())

    ref_mins.append(ref.min())
    ref_maxs.append(ref.max())
im_mins = np.array(im_mins)
im_maxs = np.array(im_maxs)
ref_mins = np.array(ref_mins)
ref_maxs = np.array(ref_maxs)

In [6]:
data
ref_data
band_num = data.shape[2]
ref1_mins= [ ]
ref1_maxs= [ ]
for idx in range(band_num):
    ref1 = ref1_data[: , :, idx]
    ref1_mins.append(ref1.min())
    ref1_maxs.append(ref1.max())
ref1_mins = np.array(ref1_mins)
ref1_maxs = np.array(ref1_maxs)

In [ ]:
import numpy as np
import plotly as py
import plotly.graph_objs as go
import pandas as pd

data_num = im_maxs.shape[0]
wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]
x_data = np.array(wavelengths)

#x_data = np.arange(0, data_num)

#df = pd.DataFrame(dict(
#    x = x_data.tolist(),
#    im_mins = im_mins.tolist(),
#    im_maxs = im_maxs.tolist(),
#    ref_mins = ref_mins.tolist(),
#    ref_maxs = ref_maxs.tolist(),
#))

type = ['im_mins']*data_num+['im_maxs']*data_num+['ref_mins']*data_num+['ref_maxs']*data_num+['ref1_mins']*data_num+['ref1_maxs']*data_num
x = x_data.tolist()*6

df = pd.DataFrame(dict(
    band = x,
    responce= im_mins.tolist() + im_maxs.tolist() + ref_mins.tolist() + ref_maxs.tolist() + ref1_mins.tolist() + ref1_maxs.tolist(),
    type= type,
))
fig = px.line(df, x="band", y="responce", color='type') 
fig.show()

#df = df.sort_values(by="x")
#fig = px.line(df, x="x", y="y", title="Sorted Input") 
#fig.show()

#  (a) wavelength-wise correction

In [ ]:
wavelengths

In [7]:
import cv2 as cv
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
def on_trackbar(idx):
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    #print(f'im min: {im.min()}, max: {im.max()}')
    #print(f'ref min: {ref.min()}, max: {ref.max()}')
    im_corrected = im/(ref1-ref)
    im_corrected /= im_corrected.max()
    print("\r", f'wavelengths: {wavelengths[idx]}', end="", flush=True)
    #print(f'corrected min: {im_corrected.min()}, max: {im_corrected.max()}')
    # seems range within 0-1.0 float, so directly show
    cv.imshow(title_window, im_corrected)
title_window = 'HSI corrected'
cv.namedWindow(title_window, 0)
slider_max = data.shape[2]
trackbar_name = f'{wavelengths_bar[0]}:{wavelengths_bar[-1]}'
cv.createTrackbar(trackbar_name, title_window , 0, slider_max, on_trackbar)
on_trackbar(0)
cv.waitKey()

 wavelengths: 637.26

-1

In [5]:
import cv2 as cv
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
def on_trackbar(idx):
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    #print(f'im min: {im.min()}, max: {im.max()}')
    #print(f'ref min: {ref.min()}, max: {ref.max()}')
    im_corrected = im/(ref1-ref)
    im_corrected /= im_corrected.max()
    print("\r", f'wavelengths: {wavelengths[idx]}', end="", flush=True)
    #print(f'corrected min: {im_corrected.min()}, max: {im_corrected.max()}')
    # seems range within 0-1.0 float, so directly show

    taget_y_0 = 119
    taget_y_1 = 290
    taget_x_0 = 162
    taget_x_1 = 350
    im_corrected = im_corrected[taget_y_0:taget_y_1, taget_x_0:taget_x_1]
    cv.imshow(title_window, im_corrected)

title_window = 'HSI corrected'
cv.namedWindow(title_window, 0)
slider_max = data.shape[2]
trackbar_name = f'{wavelengths_bar[0]}:{wavelengths_bar[-1]}'
cv.createTrackbar(trackbar_name, title_window , 0, slider_max, on_trackbar)
on_trackbar(0)
cv.waitKey()

 wavelengths: 393.8

-1

# (b) RGB color image
Red (640 nm), Green (550 nm) and Blue (460 nm) channels

In [6]:
wavelengths[373]

'891.1'

In [7]:
#tarrget_wavelengths = [640, 550, 460]
tarrget_wavelengths = [891, 880, 910]

wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]

import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

tarrget_values = []
tarrget_idxes = []
for target in tarrget_wavelengths:
    nearest_value, nearest_idx= find_nearest(wavelengths, target)
    tarrget_values.append(nearest_value)
    tarrget_idxes.append(nearest_idx)
tarrget_values, tarrget_idxes

([891.1, 880.28, 910.06], [373, 365, 387])

In [8]:
import cv2 as cv
data_num =  data.shape[2]
im_corrected_rgb = None
im_corrected_bgr = None
def on_trackbar(idx):
    im_channels =  []
    for idx_target in tarrget_idxes:
        #new_idx = idx+idx_target
        new_idx = idx+idx_target-50
        # need convert to int to avoid issues
        new_idx = int(new_idx)
        if new_idx >= data_num or new_idx < 0:
            print('new_idx is exceed, set to boundary value')
            new_idx = min(max(0, new_idx), data_num-1)
        im = data[: , :, new_idx]
        ref = ref_data[: , :, new_idx]
        ref1 = ref1_data[: , :, new_idx]
        im_corrected = im/(ref1-ref)
        im_corrected /= im_corrected.max()
        im_channels.append(im_corrected)
    im_corrected_rgb = np.concatenate(im_channels, axis=2)
    # cv.imshow use bgr for show
    im_corrected_bgr = im_corrected_rgb[:, :, ::-1]
    cv.imshow(title_window, im_corrected_bgr)
title_window = 'HSI corrected'
cv.namedWindow(title_window, 0)
#slider_max = data_num
slider_max = 100
trackbar_name = 'band %d' % slider_max
#cv.createTrackbar(trackbar_name, title_window , 0, slider_max, on_trackbar)
cv.createTrackbar(trackbar_name, title_window , 50, slider_max, on_trackbar)
#on_trackbar(0)
on_trackbar(50)
cv.waitKey()

-1

# (c) & (d) Segment samples / Extract boundaries


In [9]:
idx = 0
im_channels =  []
for idx_target in tarrget_idxes:
    new_idx = idx+idx_target
    new_idx = int(new_idx)
    im = data[: , :, new_idx]
    ref = ref_data[: , :, new_idx]
    ref1 = ref1_data[: , :, new_idx]
    im_corrected = im/(ref1- ref)
    im_channels.append(im_corrected)
im_corrected_rgb = np.concatenate(im_channels, axis=2)
# cv.imshow use bgr for show
im_corrected_bgr = im_corrected_rgb[:, :, ::-1]
cv.imshow(title_window, im_corrected_bgr)
cv.waitKey()

-1

In [45]:
im_corrected_rgb.min(), im_corrected_rgb.max(), im_corrected_rgb.shape


(0.0, 1.3851091142490373, (1000, 1600, 3))

In [ ]:
import plotly.express as px
import numpy as np
im_corrected_rgb_norm = (im_corrected_rgb - im_corrected_rgb.min()) / (im_corrected_rgb.max() - im_corrected_rgb.min())
im_corrected_rgb_norm_rescale = np.array(im_corrected_rgb_norm*255, dtype=np.uint8).squeeze()
print(im_corrected_rgb_norm_rescale.shape)
fig = px.imshow(im_corrected_rgb_norm_rescale)
fig.show()

In [ ]:
"""
Graph Cut
"""
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = im_corrected_rgb_norm_rescale
mask = np.zeros(img.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
rect = (400,0,1250,1000)
cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]
#plt.imshow(img),plt.colorbar(),plt.show()
import plotly.express as px
import numpy as np
im_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ret, im_t = cv.threshold(im_gray, 1, 255, cv.THRESH_BINARY)

fig = px.imshow(im_t)
fig.show()

In [ ]:
"""
this cell shows distribution, but need 8s to run
"""
import numpy as np
import plotly as py
import plotly.graph_objs as go
import pandas as pd

im_corrected_rgb_data = im_corrected_rgb_norm_rescale.reshape(-1, 3)
data_num = im_corrected_rgb_data.shape[0]
x_data = np.arange(0, data_num)


type = ['R']*data_num+['G']*data_num+['B']*data_num
df = pd.DataFrame(dict(
    #band = x,
    intensity= im_corrected_rgb_data[:, 0].tolist() + im_corrected_rgb_data[:, 1].tolist() + im_corrected_rgb_data[:, 2].tolist(),
    type= type,
))
fig = px.histogram(df, x="intensity", color='type', color_discrete_sequence=['red', 'green', 'blue']) 
fig.show()

In [11]:
import cv2 as cv
im = im_corrected_rgb_norm_rescale
im_gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
#im = cv.Sobel(im, cv.CV_64F, 1, 1, ksize=3)

ret, im_t = cv.threshold(im_gray, 20, 255, cv.THRESH_BINARY)
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_t)
cv.waitKey(0)

-1

In [12]:
# for convinenent use larger filter
#kernel=cv.getStructuringElement(cv.MORPH_RECT,(20,20))
kernel=cv.getStructuringElement(cv.MORPH_RECT,(5,5))
eroded=cv.erode(im_t,kernel)
kernel=cv.getStructuringElement(cv.MORPH_RECT,(3,3))
#iterations = 10
iterations = 2
im_t_dilated=cv.dilate(eroded,kernel, iterations=iterations)
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_t_dilated)
cv.waitKey(0)

-1

In [13]:
im = im_corrected_rgb_norm_rescale[:,:,::-1]
im_copy = im.copy()
contours, hierarchy = cv.findContours(image=im_t_dilated, mode=cv.RETR_EXTERNAL,
                                    method=cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(image=im_copy, contours=contours, contourIdx=-1,
                color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)
# see the results
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_copy)
cv.waitKey(0)

-1

In [14]:
im = im_corrected_rgb_norm_rescale[:,:,::-1]
im_copy = im.copy()
for contour in contours:
    color = np.random.randint(0, 256, 3).tolist()
    cv.drawContours(image=im_copy, contours=[contour], contourIdx=-1,
                    color=color, thickness=2, lineType=cv.LINE_AA)
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_copy)
cv.waitKey(0)

-1

#  (e) Number the samples

In [53]:
im = im_t_dilated
im_copy = im.copy()
font=cv.FONT_HERSHEY_DUPLEX
center_pts = []
center_refs = []
for i_contour, contour in enumerate(contours):
    x = int(contour[:, :, 0].mean())
    y = int(contour[:, :, 1].mean())
    center_refs.append(x+4*y)
    center_pts.append((x, y))
idx_sorted = np.argsort(center_refs)
for i_idx_sorted, idx in enumerate(idx_sorted):
    x, y = center_pts[idx]
    cv.putText(im_copy, str(i_idx_sorted+1) , (x-5,y+5), font, 1, 0, 2)
print(len(contours))
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_copy)
cv.waitKey(0)

18


-1

In [62]:
import cv2 as cv
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
def on_trackbar(idx):
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    im_corrected = im/(ref1-ref)
    im_corrected/=im_corrected.max()

    print("\r", f'wavelengths: {wavelengths[idx]}', end="", flush=True)
    cv.putText(im_corrected, str(wavelengths[idx]) , (50, 50), font, 1, 255, 2)
    for i_idx_sorted, idx in enumerate(idx_sorted):
        x, y = center_pts[idx]
        cv.putText(im_corrected, str(i_idx_sorted+1) , (x-100,y-70), font, 1, 255, 2)
    cv.imshow(title_window, im_corrected)
title_window = 'HSI corrected'
cv.namedWindow(title_window, 0)
slider_max = data.shape[2]
trackbar_name = f'{wavelengths_bar[0]}:{wavelengths_bar[-1]}'
cv.createTrackbar(trackbar_name, title_window , 0, slider_max, on_trackbar)
on_trackbar(0)
cv.waitKey()

 wavelengths: 916.83

-1

# (f) Calculate the pixel areas

In [60]:
im = im_t_dilated
im_copy = im.copy()
font=cv.FONT_HERSHEY_DUPLEX
center_pts = []
center_areas = []
for i_contour, contour in enumerate(contours):
    x = int(contour[:, :, 0].mean())
    y = int(contour[:, :, 1].mean())
    area = cv.contourArea(contour)
    area = round(area, 2)
    center_pts.append((x, y))
    center_areas.append(area)
for i_idx_sorted, idx in enumerate(idx_sorted):
    x, y = center_pts[idx]
    area = center_areas[idx]
    cv.putText(im_copy, str(area) , (x-50,y+5), font, 0.8, 0, 2)
cv.namedWindow(title_window, 0)
cv.imshow(title_window, im_copy)
cv.waitKey(0)

-1

In [56]:
center_areas = np.array(center_areas)
center_areas.min(), center_areas.mean(), center_areas.max()

(7340.5, 15432.361111111111, 20488.5)

In [57]:
center_areas = np.array(center_areas)
import pandas as pd
import numpy as np

df_describe = pd.DataFrame(center_areas)
df_describe.describe()

,0
count,18.000000
mean,15432.361111
std,3458.726767
min,7340.500000
25%,14707.500000
50%,16227.250000
75%,17530.875000
max,20488.500000


# (g) each sample spectra

In [58]:
"""
need 5 minutes to run
"""
import cv2 as cv
from tqdm import tqdm
wavelengths = data.metadata['wavelength']
wavelengths = [float(x) for x in wavelengths]
data_num = len(wavelengths)
peanuts_dict = {}
for idx in tqdm(range(data_num), ncols=80):
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    im_corrected = im/(ref1- ref)
    for i_idx_sorted, idx in enumerate(idx_sorted):
        contour = contours[idx]
        mask = np.zeros_like(im_corrected)
        cv.drawContours(mask, [contour], -1, 255,-1)
        pts  = np.where(mask == 255)
        roi = im_corrected[pts[0], pts[1]]
        mean_spectral = round(np.mean(roi), 4)
        #print('mean_spectral:', mean_spectral)
        if i_idx_sorted not in peanuts_dict:
            peanuts_dict[i_idx_sorted] = []
        peanuts_dict[i_idx_sorted].append(mean_spectral)
peanuts_dict.keys()

100%|█████████████████████████████████████████| 462/462 [01:25<00:00,  5.43it/s]


dict_keys([14, 12, 15, 13, 16, 17, 10, 9, 8, 7, 6, 11, 3, 0, 1, 2, 5, 4])

In [56]:
len(peanuts_dict.keys())

18

In [ ]:
import numpy as np
import plotly as py
import plotly.graph_objs as go
import pandas as pd

data_num = len(wavelengths)
x_data = np.array(wavelengths)


item_num = len(peanuts_dict.keys())

peanuts_dict_sorted=sorted(peanuts_dict.items(), key=lambda item :item[0])

type = []
responce = []
for idx, values in peanuts_dict_sorted:
    type += [str(idx+1)]*data_num
    responce += values

x = x_data.tolist()*item_num
df = pd.DataFrame(dict(
    wavelength = x,
    reflectance= responce,
    number = type,
))
fig = px.line(df, x="wavelength", y="reflectance", color='number',
                 labels={
                     "wavelength": "Wavelength (nm)",
                     "reflectance": "Reflectance (a.u.)",
                 }
) 
fig.show()

# felzenszwalb-segmentation

In [ ]:
!pip install felzenszwalb-segmentation

In [40]:
"""
need 1min for 800x1400
"""
import numpy as np
from glob import glob
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
from PIL import Image
from matplotlib import pyplot as plt
from felzenszwalb_segmentation_revised import segment
import random
import numpy as np

#import plotly.express as px
#from plotly.subplots import make_subplots
#from plotly.graph_objs import Image as plot_Image

image_files = glob(r'E:\PHD\Blueberry\Blueberry\samples\*.jpg')
im_num = len(image_files)
#fig = make_subplots(rows=1, cols=2)
#tot_ims= [ ]

norm = plt.Normalize(1,4)
cmap = plt.cm.RdYlGn
np.random.seed(0)
c = np.random.randint(1,5,size=15)

for i_im in range(im_num):
    if i_im !=2:
        continue
    random.seed(10)
    image = np.array(Image.open(image_files[i_im]))
    segmented_image = segment(image, 0.2, 400, 50)
    segmented_image = segmented_image.astype(np.uint8)

    #tot_ims.append(image)
    #tot_ims.append(segmented_image)
    #tot_im = np.concatenate([image, segmented_image], 1)
    #tot_im1 = np.concatenate([segmented_image, image], 1)
    #tot_im = np.concatenate([tot_im, tot_im1], 0)
    #fig = px.imshow(tot_im)
    #fig.update_xaxes(showticklabels = False)
    #fig.update_yaxes(showticklabels = False)
    #fig.update_traces(hoverinfo='none', hovertemplate=None)
    #fig.add_trace(px.imshow(image).data[0], row=i_im, col=1)
    #fig.add_trace(px.imshow(segmented_image).data[0], row=i_im, col=2)

    #fig1 = px.imshow(image)
    #fig2 = px.imshow(segmented_image)

    fig = plt.figure(figsize=(6, 6))
    ax1 = fig.add_subplot(1, 2, 1)
    handle1 = plt.imshow(image)
    ax2 = fig.add_subplot(1, 2, 2)
    handle2= plt.imshow(segmented_image)

    annot1 = ax1.annotate("", xy=(0,0), xytext=(40,40),textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="w"),
                        arrowprops=dict(arrowstyle="->"))
    annot1.set_visible(False)

    annot2 = ax2.annotate("", xy=(0,0), xytext=(40,40),textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="w"),
                        arrowprops=dict(arrowstyle="->"))
    annot2.set_visible(False)

    def on_move(event):
        if event.inaxes:
            x, y  = event.xdata, event.ydata
            annot1.xy = [x, y]
            x = round(x)
            y = round(y)
            text = f"{x}, {y}, {image[y, x]}"
            annot1.set_text(text)
            annot1.get_bbox_patch().set_facecolor(cmap(norm(c[0])))
            annot1.get_bbox_patch().set_alpha(0.4)
            annot1.set_visible(True)

            annot2.xy = [x, y]
            text = f"{x}, {y}, {segmented_image[y, x]}"
            annot2.set_text(text)
            annot2.get_bbox_patch().set_facecolor(cmap(norm(c[0])))
            annot2.get_bbox_patch().set_alpha(0.4)
            annot2.set_visible(True)
            fig.canvas.draw_idle()

    fig.canvas.mpl_connect("motion_notify_event", on_move)

plt.show()

#tot_ims = np.array(tot_ims)
#fig = px.imshow(tot_ims, binary_string=True, facet_col=0, facet_col_wrap=2)

#fig.update_layout(hovermode="y unified")
#fig.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across", spikethickness=0.1)
#fig.update_yaxes(showspikes=True, spikecolor="orange", spikesnap="cursor", spikemode="across", spikethickness=0.1)
#fig.update_layout(spikedistance=1000, hoverdistance=100)
#fig.show()

#fig1.update_traces(hovertemplate=None)
#fig2.update_traces(hovertemplate=None)
#fig1.update_layout(hovermode="x")
#fig2.update_layout(hovermode="x")
#fig1.show()
#fig2.show()

In [38]:
import numpy as np
from glob import glob
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
from PIL import Image
from matplotlib import pyplot as plt
from felzenszwalb_segmentation_revised import segment
import random
import numpy as np

from  pathlib import Path
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
save_dir= r'E:\PHD\Blueberry\Blueberry\one_sample_wavelength_change'
save_dir = Path(save_dir)

save_path = save_dir/'origin1'
save_path = save_path / f'*.png'

save_path1 = save_dir/'segment1'
save_path1 = save_path1 / f'*.png'

image_files = glob(save_path.__str__())
image_files1 = glob(save_path1.__str__())
image_files = sorted(image_files, key=lambda x: float(Path(x).name.split('_')[0]))
image_files1 = sorted(image_files1, key=lambda x: float(Path(x).name.split('_')[0]))
im_num = len(image_files)
im_num1 = len(image_files1)

norm = plt.Normalize(1,4)
cmap = plt.cm.RdYlGn
np.random.seed(10)
c = np.random.randint(1,5,size=15)
first_time = True
# 361 256
im_idx = 256
def compare_im_with_process(event):
    global first_time, im_idx
    if True:
    #for i_im in range(im_num):
    #    if i_im <= 364:
    #        continue
        if first_time is False:
            if event.key == 'a':
                im_idx -= 1
            if event.key == 'd':
                im_idx += 1
        i_im = im_idx
        image = np.array(Image.open(image_files[i_im]))
        segmented_image = np.array(Image.open(image_files1[i_im]))
        #segmented_image = segment(image, 0.2, 400, 50)
        #segmented_image = segmented_image.astype(np.uint8)

        fig = plt.figure(0, figsize=(6, 6))
        fig.clear()
        ax1 = fig.add_subplot(1, 2, 1)
        handle1 = plt.imshow(image)
        ax2 = fig.add_subplot(1, 2, 2)
        handle2= plt.imshow(segmented_image)
        ax1.set_title(wavelengths[i_im])

        annot1 = ax1.annotate("", xy=(0,0), xytext=(40,40),textcoords="offset points",
                            bbox=dict(boxstyle="round", fc="w"),
                            arrowprops=dict(arrowstyle="->"))
        annot1.set_visible(False)

        annot2 = ax2.annotate("", xy=(0,0), xytext=(40,40),textcoords="offset points",
                            bbox=dict(boxstyle="round", fc="w"),
                            arrowprops=dict(arrowstyle="->"))
        annot2.set_visible(False)

        def on_move(event):
            if event.inaxes:
                x, y  = event.xdata, event.ydata
                annot1.xy = [x, y]
                x = round(x)
                y = round(y)
                text = f"{x}, {y}, {image[y, x]}"
                annot1.set_text(text)
                annot1.get_bbox_patch().set_facecolor(cmap(norm(c[0])))
                annot1.get_bbox_patch().set_alpha(0.4)
                annot1.set_visible(True)

                annot2.xy = [x, y]
                text = f"{x}, {y}, {segmented_image[y, x]}"
                annot2.set_text(text)
                annot2.get_bbox_patch().set_facecolor(cmap(norm(c[0])))
                annot2.get_bbox_patch().set_alpha(0.4)
                annot2.set_visible(True)
                fig.canvas.draw_idle()
            else:
                annot1.set_visible(False)
                annot2.set_visible(False)
                fig.canvas.draw_idle()

        fig.canvas.mpl_connect("motion_notify_event", on_move)
        if first_time is True:
            cid = fig.canvas.mpl_connect('key_press_event', compare_im_with_process)
        if first_time is True:
            plt.show()
            first_time = False
        else:
            plt.show(block=False)
compare_im_with_process(0)

In [2]:
from felzenszwalb_segmentation_revised import segment
import random
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
title_window = 'HSI corrected'
title_window1 = 'processed'
font=cv.FONT_HERSHEY_DUPLEX

import cv2 as cv
import random
wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
title_window = 'HSI corrected'
title_window1 = 'processed'
font=cv.FONT_HERSHEY_DUPLEX

from random import randint

taget_y_0 = 119
taget_y_1 = 290
taget_x_0 = 162
taget_x_1 = 350

def get_random_rgb_image():
    rgb = np.zeros(3, dtype=int)
    rgb[0] = randint(0, 255)
    rgb[1] = randint(0, 255)
    rgb[2] = randint(0, 255)
    return rgb

height = taget_y_1 - taget_y_0
width = taget_x_1 - taget_x_0
colors = np.zeros(shape=(height * width, 3))
for i in range(height * width):
    colors[i, :] = get_random_rgb_image()

def on_trackbar(idx):
    random.seed(10)
    np.random.seed(10)
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    im_corrected = im/(ref1-ref)
    im_corrected/=im_corrected.max()
    print("\r", f'wavelengths: {wavelengths[idx]}', end="", flush=True)
    taget_y_0 = 119
    taget_y_1 = 290
    taget_x_0 = 162
    taget_x_1 = 350
    im_corrected = (im_corrected*255).astype(np.uint8)
    im_corrected = im_corrected[taget_y_0:taget_y_1, taget_x_0:taget_x_1]
    im_corrected = np.concatenate([im_corrected,im_corrected,im_corrected], -1)
    #print(im_corrected.shape)
    segmented_image = segment(im_corrected, 0.2, 400, 50, colors=colors)
    segmented_image = segmented_image.astype(np.uint8)

    #cv.putText(im_corrected, str(wavelengths[idx]) , (3, 3), font, 1, 255, 2)
    #for i_contour, idx in enumerate(idx_sorted):
    #    x, y = center_pts[idx]
    #    cv.putText(im_corrected, str(i_contour+1) , (x-100,y-70), font, 1, 255, 2)
    cv.imshow(title_window, im_corrected)
    cv.imshow(title_window1, segmented_image)
cv.namedWindow(title_window, 0)
cv.namedWindow(title_window1, 0)
slider_max = data.shape[2]
trackbar_name = f'{wavelengths_bar[0]}:{wavelengths_bar[-1]}'
cv.createTrackbar(trackbar_name, title_window, 378, slider_max, on_trackbar)
#cv.createTrackbar(trackbar_name, title_window1 , 0, slider_max, on_trackbar)
cv.waitKey()

 wavelengths: 897.86

e:\Repo\Biang\Graphics\Blueberry\felzenszwalb_segmentation_revised\utils\filter_utils.py:14: RuntimeWarning: overflow encountered in ubyte_scalars
  src[y, max(x - i, 0)] + src[y, min(x + i, width - 1)])


-1

In [7]:
import cv2 as cv
from felzenszwalb_segmentation_revised import segment
from  pathlib import Path
import random

from random import randint

taget_y_0 = 119
taget_y_1 = 290
taget_x_0 = 162
taget_x_1 = 350

def get_random_rgb_image():
    rgb = np.zeros(3, dtype=int)
    rgb[0] = randint(0, 255)
    rgb[1] = randint(0, 255)
    rgb[2] = randint(0, 255)
    return rgb

height = taget_y_1 - taget_y_0
width = taget_x_1 - taget_x_0
colors = np.zeros(shape=(height * width, 3))
for i in numpy save arrayrange(height * width):
    colors[i, :] = get_random_rgb_image()

wavelengths = data.metadata['wavelength']
wavelengths_bar = [round(float(x)) for x in wavelengths]
title_window = 'HSI corrected'
title_window1 = 'processed'
font=cv.FONT_HERSHEY_DUPLEX

save_dir= r'E:\PHD\Blueberry\Blueberry\one_sample_wavelength_change'
save_dir = Path(save_dir)
wavelength_num = len(wavelengths)

cv.namedWindow(title_window, 0)
cv.namedWindow(title_window1, 0)

for idx in range(wavelength_num):
    random.seed(10)
    im = data[: , :, idx]
    ref = ref_data[: , :, idx]
    ref1 = ref1_data[: , :, idx]
    im_corrected = im/(ref1-ref)
    im_corrected/=im_corrected.max()

    print("\r", f'wavelengths: {wavelengths[idx]}', end="", flush=True)
    taget_y_0 = 119
    taget_y_1 = 290
    taget_x_0 = 162
    taget_x_1 = 350
    im_corrected = (im_corrected*255).astype(np.uint8)
    im_corrected = im_corrected[taget_y_0:taget_y_1, taget_x_0:taget_x_1]
    im_corrected = np.concatenate([im_corrected,im_corrected,im_corrected], -1)
    segmented_image = segment(im_corrected, 0.2, 400, 50, colors=colors)
    segmented_image = segmented_image.astype(np.uint8)

    cv.imshow(title_window, im_corrected)
    cv.imshow(title_window1, segmented_image)
    
    save_path = save_dir/'origin'
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = save_path / f'{idx}_{wavelengths[idx]}.png'
    #print(save_path)
    cv.imwrite(save_path.__str__(), im_corrected)

    save_path = save_dir/'segment'
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = save_path / f'{idx}_{wavelengths[idx]}.png'
    cv.imwrite(save_path.__str__(), segmented_image)
    cv.waitKey(1)

 wavelengths: 1010.64

# Others


check ref data

In [ ]:
from modeling_spectral_data import find_nearest

band, idx = find_nearest(wavelengths, 680)
print(band, idx)
idx = int(idx)
#ref = ref_data[200:, :, idx]
ref = ref_data[:1600, :, idx]
im1 = data[:, :, idx]
im1 = im1 / ref

band, idx = find_nearest(wavelengths, 900)
print(band, idx)
idx = int(idx)
#ref = ref_data[200:, :, idx]
ref = ref_data[:1600, :, idx]
im2 = data[:, :, idx]
im2 = im2 / ref

diff_im = im2-im1
diff_im = diff_im[:, :, 0]
print(diff_im.min(), diff_im.max())

data distribution

In [1]:
from modeling_spectral_data import load_hyper_spectral_data, good_dir, bad_dir, good_dir1, bad_dir1
from modeling_spectral_data import get_calibrated_image

"""
ref_data
1st day 1800

2nd day 1600
    ref_data[:1600, :, 0]
"""

hyper_spectral_data_dict, data_idx_dict = load_hyper_spectral_data()
ref_data = hyper_spectral_data_dict['ref_data']

In [6]:
data_names = ['data_bad', 'data_bad1', 'data_bad2', 'data_bad3', 'data_bad4']
#data_names = ['data_good', 'data_good1', 'data_good2', 'data_good3', 'data_good4']

In [3]:
def get_data(data_name, data_idx_dict):
    data = data_idx_dict[data_name]['data']
    sample_start_idx = data_idx_dict[data_name]['start_idx']
    save_name = data_idx_dict[data_name]['name']

    wavelengths = data.metadata['wavelength']
    wavelengths = [float(x) for x in wavelengths]
    data_num = len(wavelengths)
    title_window = 'x'

    print(data_name, save_name)
    return data, wavelengths

In [7]:
im_corrected_datas = []
for data_name in data_names:
    data, wavelengths = get_data(data_name, data_idx_dict)
    im1 = get_calibrated_image(680, wavelengths, data, ref_data)
    im2 = get_calibrated_image(900, wavelengths, data, ref_data)
    diff_im = im2-im1
    diff_im = diff_im[:, :, 0]
    im_corrected_datas.append(diff_im)


data_bad bad_1_42
679.96 216
900.58 380
data_bad1 bad_43_84
679.96 216
900.58 380
data_bad2 bad_85_126
679.96 216
900.58 380
data_bad3 bad_127_168
679.96 216
900.58 380
data_bad4 bad_169_210
679.96 216
900.58 380


In [8]:
from modeling_spectral_data import show_single_spectral_im_pixel_distribution
show_single_spectral_im_pixel_distribution(im_corrected_datas, 'bad')

bad_0 -0.6053791017294667 1.129282140107775
bad_1 -0.6072484012791934 1.1620011186489607
bad_2 -0.6119473020875213 1.1504449388209121
bad_3 -0.8425890560945177 1.1052668724310515
bad_4 -0.7348261794109441 0.95702395081425
F:\test\xxx.html
